In [1]:
import os
import time
import pickle

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from retrying import retry
from urllib.parse import urlparse, parse_qs

In [3]:
import unidecode
import re
from urllib.parse import urljoin
from dataclasses import dataclass, field, asdict
from datetime import date
from typing import List, Optional
import json

In [4]:
from utils.scrap_data import SEASON_DATA, CLUB_DATA

In [5]:
from datetime import datetime

PL_MONTHS = {
    "stycznia": 1,
    "lutego": 2,
    "marca": 3,
    "kwietnia": 4,
    "maja": 5,
    "czerwca": 6,
    "lipca": 7,
    "sierpnia": 8,
    "września": 9,
    "października": 10,
    "listopada": 11,
    "grudnia": 12,
}

In [6]:
@dataclass
class Transfer:
    season: str
    date: str
    left: str
    joined: str
    mv: str
    fee: str

In [7]:
@dataclass
class TransfermarktPlayer:
    player_id: int
    name: str
    profile_url: str
    
    date_of_birth: Optional[date] = None
    position: Optional[str] = None
    transfers: Optional[List[Transfer]] = field(default_factory=list)
    id_90minut: Optional[int] = None

In [11]:
tstest = TransfermarktScrapper()

In [13]:
buksa = tstest.extract_player_ids_from_search('Adam Buksa')

In [20]:
buksa[0].player_id

267160

In [21]:
tstest.enrich_with_transfers(buksa[0])

In [22]:
buksa

[TransfermarktPlayer(player_id=267160, name='Adam Buksa', profile_url='https://www.transfermarkt.de/adam-buksa/profil/spieler/267160', date_of_birth=None, position=None, transfers=[Transfer(season='25/26', date='26.08.2025', left='FC Midtjylland', joined='Udinese', mv='6,00 Mio. €', fee='5,00 Mio. €'), Transfer(season='24/25', date='11.07.2024', left='RC Lens', joined='FC Midtjylland', mv='6,00 Mio. €', fee='4,00 Mio. €'), Transfer(season='23/24', date='30.06.2024', left='Antalyaspor', joined='RC Lens', mv='6,00 Mio. €', fee='Leih-Ende'), Transfer(season='23/24', date='18.07.2023', left='RC Lens', joined='Antalyaspor', mv='5,00 Mio. €', fee='Leihe'), Transfer(season='22/23', date='01.07.2022', left='New England', joined='RC Lens', mv='9,00 Mio. €', fee='6,00 Mio. €'), Transfer(season='19/20', date='01.01.2020', left='Pogon Stettin', joined='New England', mv='1,00 Mio. €', fee='4,20 Mio. €'), Transfer(season='18/19', date='01.07.2018', left='Zaglebie Lubin', joined='Pogon Stettin', mv='

In [8]:
class TransfermarktScrapper():
    BASE_URL = 'https://www.transfermarkt.de'
    QUERY_URL = BASE_URL+'/schnellsuche/ergebnis/schnellsuche?query='
    TRANSFER_URL = BASE_URL+r'/ceapi/transferHistory/list/'
    HEADERS = {"User-Agent": "Mozilla/5.0"}
    def __init__(self):
        pass

    @staticmethod
    def _normalize_name(name: str) -> str:
        ''' 
        Normalize name for a search request
        '''
        name = unidecode.unidecode(name.lower())
        name = re.sub(r'[^a-z\s-]', '', name)
        #name = re.sub(r'[-]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.replace(' ', '+').strip()
        
    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def extract_player_ids_from_search(self, name_raw: str) -> list[TransfermarktPlayer]:
        name_formatted = self._normalize_name(name_raw)
        resp = requests.get(self.QUERY_URL+name_formatted, headers=self.HEADERS)
        soup = BeautifulSoup(resp.text, "lxml")
    
        table = soup.find("table", class_="items")
        players = []
        
        if not table:
            return []
    
        for row in table.tbody.find_all("tr", recursive=False):
            name_cell = row.find("td", class_="hauptlink")
            if not name_cell:
                continue
    
            a = name_cell.find("a", href=True)
            if not a:
                continue
    
            href = a["href"]
            player_id = href.rstrip("/").split("/")[-1]
    
            players.append(TransfermarktPlayer(
                player_id = int(player_id),
                name = a.get_text(strip=True),
                profile_url = urljoin(self.BASE_URL, href),
            ))
    
        return players

    @staticmethod
    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def enrich_with_dob_and_position(player: TransfermarktPlayer) -> None:
        resp = requests.get(player.profile_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")
        meta = soup.find("meta", attrs={"name": "description"})
        
        if not meta:
            pass
        else:
            dates = re.findall(r"(\d{2}\.\d{2}\.\d{4})", meta['content'])
            if len(dates) > 0:
                player.date_of_birth = min(dates, key=lambda d: datetime.strptime(d, '%d.%m.%Y'))
            
            parts = [p.strip() for p in meta['content'].split("➤")]
            if len(parts) >= 3:
                player.position = parts[2]

    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def enrich_with_transfers(self, player: TransfermarktPlayer) -> None:
        resp = requests.get(self.TRANSFER_URL+str(player.player_id), headers=self.HEADERS).text
        resp_json = json.loads(resp)
        for val in resp_json["transfers"]:
            player.transfers.append(
                Transfer(
                season = val["season"],
                date = val["date"],
                fee = val["fee"],
                mv = val["marketValue"],
                joined =  val["to"]["clubName"],
                left =  val["from"]["clubName"]
                ))


In [9]:
class Scrapper():
    def __init__(self):
        pass

    @staticmethod
    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def extract_table(link: str = None, league_id: int = None, url_var: int = None) -> pd.DataFrame:
        '''Template URL: http://www.90minut.pl/liga/YYY/ligaXXX.html
        XXX refers to league ID
        YYY refers to the variable in URL (until season 2018/2019 -> 0, after that season -> 1)
        Example URL: Template URL: http://www.90minut.pl/liga/0/liga6826.html'''
        
        time.sleep(3)
        if link is not None:
            resp = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        else:
            url = f'http://www.90minut.pl/liga/{url_var}/liga{league_id}.html'
            resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")
        tables = soup.find_all('table')
        table = pd.read_html(StringIO(str(tables[3])), header = 3, index_col = 0)[0].iloc[:,:7].dropna(how = 'all')
        table = table[table['Nazwa'].isin(CLUB_DATA.keys()) == True] #keeping only records containing club names in the club name column
        table = table.astype({'M.': 'int32', 'Z.': 'int32', 'R.': 'int32', 'P.': 'int32'})
        table['Pkt.'] = table['Pkt.'].apply(lambda x : str(x))
        return table

    @staticmethod
    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def extract_team_season_stats(link: str = None, club_id: int = None, season_id: int = None, league_id: int = None) -> list[pd.DataFrame, list]:
        '''Template URL: http://www.90minut.pl/bilans.php?id_klub=XXX&id_sezon=YY&id=ZZZ
        XXX refers to league ID
        YY refers to season ID
        ZZ refers to league ID
        Example URL: http://www.90minut.pl/bilans.php?id_klub=171&id_sezon=83&id=6826
        
        Returns 2-element list:
        1st element: all players stats
        2nd element: dict of players names and links to profile'''

        time.sleep(3)
        if link is not None:
            resp = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        else:
            url = f'http://www.90minut.pl/bilans.php?id_klub={club_id}&id_sezon={season_id}&id={league_id}'
            resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")
        tables = soup.find_all('table')
        table_main = pd.read_html(StringIO(str(tables[3])), header = 2, index_col = 0)[0].iloc[:-1,:4].dropna(how = 'all')
        table_main = table_main.astype({'w.': 'int32', 'pełne' : 'int32', 'z ławki': 'int32', 'czas': 'int32'})
        links_list = pd.read_html(StringIO(str(tables[3])), header = 2, extract_links = 'all')[0].iloc[:-2,0].dropna(how = 'all').to_list()
        links_dict = {el[0] : el[1] for el in links_list}
        return [table_main, links_dict]

    @staticmethod
    @retry(stop_max_attempt_number=5, wait_random_min=10000, wait_random_max=20000) #5 retries, wait between 10-20 seconds between each retry
    def extract_player_information(link: str = None, player_id: int = None, season_id: int = None) -> dict:
        '''Template URL: http://www.90minut.pl/wystepy.php?id=XXX&id_sezon=ZZ
        XXX refers to player ID
        ZZZ refers to season ID
        Example URL: http://www.90minut.pl/wystepy.php?id=8311&id_sezon=83'''

        time.sleep(3)
        if link is not None:
            resp = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        else:
            url = f'http://www.90minut.pl/wystepy.php?id={player_id}&id_sezon={season_id}'
            resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")
        tables = soup.find_all('table')
        res = pd.read_html(StringIO(str(tables[0])), header = 0, index_col = 1)[0].iloc[:,1:].transpose().dropna(how = 'all')
        res_dict = {'country' : res.loc[:, 'Kraj'].iat[0],
                    'date_of_birth' : ' '.join(res.loc[:, 'Data urodzenia'].iat[0].split()[:3]),
                    'year_of_birth' : res.loc[:, 'Data urodzenia'].iat[0].split()[2]}
        return res_dict

In [10]:
class DataManipulator():
    def __init__(self):
        self.season_data_path = '../data/season_data.parquet'
        self.player_season_data_path = '../data/player_season_data.parquet'
        self.player_personal_data_path = '../data/player_personal_data.pickle'
        self.player_final_data_path = '../data/player_final_data.parquet'
        self.player_final_data_season_level_path = '../data/player_final_data_season_level.parquet'
        self.transfermarkt_player_data = '../data/transfermarkt_player_data.json'
        self.transfermarkt_player_data_no_match = '../data/transfermarkt_player_data_no_match.json'
        self.scrapper = Scrapper()
        self.t_scrapper = TransfermarktScrapper()

    @staticmethod
    def _save_pickle(file_object, path):
        with open(path, 'wb') as file:
            pickle.dump(file_object, file)
    
    @staticmethod
    def _load_pickle(path):
        with open(path, 'rb') as file:
            a = pickle.load(file)
            return a

    def _save_json(self, data, path):
        with open(path, "w", encoding="utf-8") as f:
            json.dump([asdict(p) for p in data], f, ensure_ascii=False, indent=2)
        print(f'File saved succesfully. Path: {path}')

    @staticmethod
    def _parse_player(data):
        data["transfers"] = [Transfer(**t) for t in data["transfers"]]
        return TransfermarktPlayer(**data)
    
    def _load_json(self, path):
        with open(path, "r", encoding="utf-8") as f:
            data = [self._parse_player(p) for p in json.load(f)]
        return data
        
    @staticmethod
    def _get_player_id(link: str) -> int:
        '''Extracts player ID from the player link
        Example: http://www.90minut.pl/wystepy.php?id=44893&id_sezon=103
        Player ID: 44893
        '''
        query = parse_qs(urlparse(link).query)
        return int(query['id'][0])
        
    def generate_season_data(self, overwrite = False) -> pd.DataFrame:
        '''Generates season_data.parquet file if it does not exist.
        If it exists, it checks whether all seasons are there.
        If yes, it doesn't do anything.'''

        if os.path.isfile(self.season_data_path) == False or \
        (os.path.isfile(self.season_data_path) == True and overwrite == True):
            final_df = pd.DataFrame()
            for k, v in SEASON_DATA.items():
                print(f'Extracting season: {k} data')
                tmp_df = self.scrapper.extract_table(league_id = v['LeagueID'], url_var = v['UrlVar'])
                tmp_df['Sezon'] = k
                final_df = pd.concat([final_df, tmp_df])
            print('Extraction completed.')
            final_df = final_df.astype({'Sezon': 'category'})
            final_df.index = final_df.index.map(str)
            final_df.to_parquet(self.season_data_path, engine = 'fastparquet')
            print(f'Data saved in the following path: {self.season_data_path}')

    def season_data_completness_check(self):
        if os.path.isfile(self.season_data_path) == True:
            print(f'File {self.season_data_path} already exists. Checking data completness')
            data_df = pd.read_parquet(self.season_data_path, engine = 'fastparquet')
            for season, value in SEASON_DATA.items():
                print(f'''Season: {season}, expected number of teams: {value['TeamsNumber']}, actual number of teams: {data_df[data_df['Sezon'] == season].shape[0]}''')
                if data_df[data_df['Sezon'] == season].shape[0] == value['TeamsNumber']:
                    print(f'Season: {season}, all ok.')
                else:
                    print(f'Season: {season} something is wrong, fix the data.')
        else:
            print('Season data file does not exist.')

    def generate_player_data(self) -> pd.DataFrame:
        '''Generates player_data.parquet file
        Requires season_data.parquet prior to be generated. If there is no it generates one.'''

        print('Checking for season_data.parquet')
        if os.path.isfile(self.season_data_path):
            print('File exists. Loading it.')
            clubs_data =  pd.read_parquet(self.season_data_path, engine = 'fastparquet')
        else:
            print('File does not exists. Generating one.')
            self.generate_season_data()
            clubs_data =  pd.read_parquet(self.season_data_path, engine = 'fastparquet')
        
        #loading season player data df
        if os.path.isfile(self.player_season_data_path):
            player_season_df = pd.read_parquet(self.player_season_data_path, engine = 'fastparquet')
        else:
            player_season_df = pd.DataFrame(columns=['w.', 'pełne', 'z ławki', 'czas', 'Sezon', 'Klub', 'ID'])
            player_season_df = player_season_df.astype({'w.': 'int32', 'pełne' : 'int32', 'z ławki': 'int32', 'czas': 'int32', 'ID': 'int32'})
            player_season_df = player_season_df.astype({'Sezon': 'category', 'Klub': 'category'})
        
        #loading personal player data dict
        if os.path.isfile(self.player_personal_data_path):
            player_personal_dict = self._load_pickle(self.player_personal_data_path)
        else:
            player_personal_dict = {}
            
        #iterating thhrough seasons
        for k,v in SEASON_DATA.items():
            print(f'*** Scrapping season: {k}')
            season_clubs = clubs_data[clubs_data['Sezon'].isin([k]) == True]['Nazwa'].to_list()
            #iterating through clubs
            for club in season_clubs:
                if len(player_season_df[(player_season_df['Sezon'] == k) & (player_season_df['Klub'] == club)].index) > 0:
                    print(f'Club: {club} in season: {k} has been already scrapped')
                    continue
                else:
                    print(f'** Scrapping club: {club} in season: {k}')
                    team_df, player_links = self.scrapper.extract_team_season_stats(club_id = CLUB_DATA[club], season_id = v['SeasonID'], league_id = v['LeagueID'])
                    team_df['Sezon'] = k
                    team_df['Klub'] = club
                    team_df = team_df.astype({'Sezon': 'category', 'Klub': 'category'})
                    #iterating through players
                    for player in team_df.index:
                        player_id = self._get_player_id(player_links[player])
                        team_df.at[player, 'ID'] = player_id
                        if player_id in player_personal_dict.keys():
                            print(f'Player: {player}, ID: {player_id} has been already scrapped')
                            continue
                        else:
                            print(f'* Scrapping player: {player}, ID: {player_id}')
                            player_dict = self.scrapper.extract_player_information(link = 'http://www.90minut.pl'+player_links[player])
                            player_dict['Imie i nazwisko'] = player
                            player_personal_dict[player_id] = player_dict
                        
                        print('Saving players personal data')
                        self._save_pickle(player_personal_dict, self.player_personal_data_path)
                    
                    player_season_df = pd.concat([player_season_df, team_df])
                    print('Saving players season data')
                    player_season_df.to_parquet(self.player_season_data_path, engine = 'fastparquet')
                
        print('Scrapping completed')
        print(f'Player season data saved in the following path: {self.player_season_data_path}')
        player_season_df.to_parquet(self.player_season_data_path, engine = 'fastparquet')
        print(f'Player personal data saved in the following path: {self.player_personal_data_path}')
        self._save_pickle(player_personal_dict, self.player_personal_data_path)
        
    def merge_season_player_data(self, season_agg = False):
        '''Generating dataframe containing combined season and personal players data.
        Requires both player_season_data.parquet and player_personal_data.pickle files.
        
        season_agg: if True, ignores club information and aggregates information at a season level.
        For example, if one player played in two clubs during one season, metrics from both
        clubs will be aggregated at a season level.'''

        season = pd.read_parquet(self.player_season_data_path, engine = 'fastparquet')
        if season_agg:
            season = season.reset_index(drop = False, names = 'Imie i Nazwisko')
            season = season[['Sezon', 'ID', 'Imie i Nazwisko', 'w.', 'pełne', 'z ławki', 'czas']].groupby(['Sezon', 'ID', 'Imie i Nazwisko']).sum()
            season = season.reset_index()
        else:
            season = season.reset_index(names = 'Imie i Nazwisko')

        personal = self._load_pickle(self.player_personal_data_path)

        season.loc[: ,'Polskie Obywatelstwo'] = False
        season.loc[: ,'Wiek'] = 0
        season.loc[: ,'Status Młodzieżowca'] = False
        season.loc[: ,'Przepis Aktywny'] = 'przed'
        season.loc[:, 'Poniżej 900 minut'] = False
        season.loc[:, 'Powyżej 900 minut'] = False
        season.loc[:, 'Pełne 10 meczów'] = False
        season.loc[:, 'Zmieniony 10 meczów'] = False
        season.loc[:, 'Wszedł z ławki 10 meczow'] = False
        season.loc[:, 'Powyżej 1620 minut'] = False
        season.loc[:, 'Pełne 18 meczów'] = False
        season.loc[:, 'Zmieniony 18 meczów'] = False
        season.loc[:, 'Wszedł z ławki 18 meczow'] = False
        season.loc[:, 'Młodzieżowiec ostatni sezon'] = False
        season.loc[:, 'Senior pierwszy sezon'] = False

        for idx in season.index:
            print(f'Processing row no {str(idx)} out of {len(season.index)}')
            season.at[idx, 'Wiek'] = int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth'])
            
            if 'Polska' in personal[season.at[idx, 'ID']]['country']:
                season.at[idx, 'Polskie Obywatelstwo'] = True
            
            if season.at[idx, 'Sezon'] != '2019/2020' and int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth']) <= 22 and season.at[idx, 'Polskie Obywatelstwo'] == True:
                season.at[idx, 'Status Młodzieżowca'] = True

            if season.at[idx, 'Sezon'] == '2019/2020' and int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth']) <= 23 and season.at[idx, 'Polskie Obywatelstwo'] == True:
                season.at[idx, 'Status Młodzieżowca'] = True

            if season.at[idx, 'Sezon'] in ('2019/2020', '2020/2021', '2021/2022', '2022/2023', '2023/2024', '2024/2025'):
                season.at[idx, 'Przepis Aktywny'] = 'w trakcie'

            if season.at[idx, 'czas'] < 900:
                season.at[idx, 'Poniżej 900 minut'] = True
                
            if season.at[idx, 'czas'] >= 900:
                season.at[idx, 'Powyżej 900 minut'] = True

            if season.at[idx, 'pełne'] >= 10:
                season.at[idx, 'Pełne 10 meczów'] = True

            if season.at[idx, 'w.'] - season.at[idx, 'pełne'] - season.at[idx, 'z ławki'] >= 10:
                season.at[idx, 'Zmieniony 10 meczów'] = True

            if season.at[idx, 'z ławki'] >= 10:
                season.at[idx, 'Wszedł z ławki 10 meczow'] = True
                
            if season.at[idx, 'czas'] >= 1620:
                season.at[idx, 'Powyżej 1620 minut'] = True

            if season.at[idx, 'pełne'] >= 18:
                season.at[idx, 'Pełne 18 meczów'] = True

            if season.at[idx, 'w.'] - season.at[idx, 'pełne'] - season.at[idx, 'z ławki'] >= 18:
                season.at[idx, 'Zmieniony 18 meczów'] = True

            if season.at[idx, 'z ławki'] >= 18:
                season.at[idx, 'Wszedł z ławki 18 meczow'] = True
            
            if season.at[idx, 'Sezon'] != '2018/2019' and int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth']) == 22 and season.at[idx, 'Polskie Obywatelstwo'] == True:
                season.at[idx, 'Młodzieżowiec ostatni sezon'] = True

            if season.at[idx, 'Sezon'] == '2019/2020' and int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth']) == 23 and season.at[idx, 'Polskie Obywatelstwo'] == True:
                season.at[idx, 'Młodzieżowiec ostatni sezon'] = True

            if season.at[idx, 'Sezon'] != '2019/2020' and int(season.at[idx, 'Sezon'].split(r'/')[1]) - int(personal[season.at[idx, 'ID']]['year_of_birth']) == 23 and season.at[idx, 'Polskie Obywatelstwo'] == True:
                season.at[idx, 'Senior pierwszy sezon'] = True

            if not season_agg:    
                if season.at[idx, 'Klub'] == 'Termalica Bruk-Bet Nieciecza':
                    season.at[idx, 'Klub'] = 'Bruk-Bet Termalica Nieciecza'
                
        print('Merging completed')
        if season_agg:
            season.to_parquet(self.player_final_data_season_level_path, engine = 'fastparquet')
            print(f'Final data saved in the following path: {self.player_final_data_season_level_path}')
        else:
            season.to_parquet(self.player_final_data_path, engine = 'fastparquet')
            print(f'Final data saved in the following path: {self.player_final_data_path}')

    @staticmethod
    def _parse_polish_date(text: str) -> str:
        day_str, month_pl, year_str = text.split()
        day = int(day_str)
        month = PL_MONTHS[month_pl.lower()]
        year = int(year_str)
    
        return f"{day:02d}.{month:02d}.{year}"

    def map_transfermarkt_data(self):
        if os.path.exists(self.player_personal_data_path) and os.path.exists(self.player_final_data_season_level_path):
            print('Loading 90minut player data')
            dob_90mins = dm._load_pickle(self.player_personal_data_path)
            all_players = pd.read_parquet(self.player_final_data_season_level_path, engine = 'fastparquet')
            young_players_df = all_players[all_players['Status Młodzieżowca'] == True][['ID', 'Imie i Nazwisko']].drop_duplicates()
        else:
            print('No 90 minutes data.')
            return None

        if os.path.exists(self.transfermarkt_player_data):
            print('Loading transfermarkt scrapped data')
            final_players_list = dm._load_json(self.transfermarkt_player_data)
            scrapped_ids = [e.id_90minut for e in final_players_list]
            no_match_results = []
        else:
            print('No transfermarkt data has been generated yet.')
            final_players_list = []
            scrapped_ids = []
            no_match_results = []

        time.sleep(3)
        for idx in young_players_df.index:
            name = young_players_df.at[idx, 'Imie i Nazwisko']
            id_90minut = young_players_df.at[idx, 'ID']
            dob = self._parse_polish_date(dob_90mins[id_90minut]['date_of_birth'])

            if id_90minut in scrapped_ids:
                print(f'Player: {name} information already scrapped')
                continue
                
            print(f'*** Searching for: {name}')
            search_results = self.t_scrapper.extract_player_ids_from_search(name)

            if len(search_results) == 0:
                print('No results found')
                no_match_results.append(
                TransfermarktPlayer(
                    player_id = None,
                    name = None,
                    profile_url = None,
                    id_90minut = id_90minut
                    )
                )
                self._save_json(no_match_results, self.transfermarkt_player_data_no_match)
                continue
            
            print(f'{len(search_results)} results found')
            print('Adding dob and position')
            for result in search_results:
                time.sleep(1)
                self.t_scrapper.enrich_with_dob_and_position(result)

            print('Matching with 90minut data')
            is_match = False
            for result in search_results:
                if dob == result.date_of_birth:
                    print('Match found. Generating transfer data.')
                    time.sleep(1)
                    self.t_scrapper.enrich_with_transfers(result)
                    result.id_90minut = int(id_90minut)
                    final_players_list.append(result)
                    is_match = True
                    self._save_json(final_players_list, self.transfermarkt_player_data)
            if is_match == False:
                for result in search_results:
                    print(f'No match found for player: {name}')
                    time.sleep(1)
                    self.t_scrapper.enrich_with_transfers(result)
                    result.id_90minut = int(id_90minut)
                    no_match_results.append(result)
                    self._save_json(no_match_results, self.transfermarkt_player_data_no_match)

In [25]:
dm = DataManipulator()

In [ ]:
dm.generate_season_data(overwrite = True)

In [ ]:
dm.season_data_completness_check()

In [ ]:
dm.generate_player_data()

In [ ]:
dm.merge_season_player_data()

In [ ]:
dm.merge_season_player_data(season_agg = True)

In [119]:
dm.map_transfermarkt_data()

Loading 90minut player data
Loading transfermarkt scrapped data
Player: Piotr Mroziński information already scrapped
Player: Damian Dąbrowski information already scrapped
Player: Wojciech Golla information already scrapped
Player: Michał Chrapek information already scrapped
Player: Jan Pawłowski information already scrapped
Player: Rafał Janicki information already scrapped
Player: Marcin Kamiński information already scrapped
Player: Michał Czekaj information already scrapped
Player: Dominik Kun information already scrapped
Player: Maciej Krakowiak information already scrapped
Player: Michał Żyro information already scrapped
Player: Wojciech Lisowski information already scrapped
Player: Michał Efir information already scrapped
Player: Jakub Szumski information already scrapped
Player: Bartosz Bereszyński information already scrapped
Player: Mateusz Kupczak information already scrapped
Player: Michał Kopczyński information already scrapped
Player: Dominik Furman information already scra

#### adding 3 players manually - date of birth match did not work

In [83]:
transfermarkt_data_json = dm._load_json('../data/transfermarkt_player_data.json')

In [84]:
transfermarkt_data_no_match_json = dm._load_json('../data/transfermarkt_player_data_no_match.json')

In [86]:
data_90minut = dm._load_pickle('../data/player_personal_data.pickle')

In [87]:
data_90minut[23170]

{'country': 'Polska',
 'date_of_birth': '22 marca 1995',
 'year_of_birth': '1995',
 'Imie i nazwisko': 'Hubert Laskowski'}

In [88]:
data_90minut[24217]

{'country': 'Polska',
 'date_of_birth': '3 marca 1995',
 'year_of_birth': '1995',
 'Imie i nazwisko': 'Jakub Okuszko'}

In [90]:
for player in transfermarkt_data_no_match_json:
    transfermarkt_data_json.append(player)

In [91]:
len(transfermarkt_data_json)

698

In [92]:
dm._save_json(transfermarkt_data_json, '../data/transfermarkt_player_data.json')

File saved succesfully. Path: ../data/transfermarkt_player_data.json


#### position extraction validation

In [93]:
pos_check = {}

In [95]:
for player in transfermarkt_data_json:
    if player.position in pos_check.keys():
        pos_check[player.position] += 1
    else:
        pos_check[player.position] = 1

In [96]:
pos_check.keys()

dict_keys(['Rechter Verteidiger', 'Defensives Mittelfeld', 'Innenverteidiger', 'Offensives Mittelfeld', 'Mittelstürmer', 'Torwart', 'Zentrales Mittelfeld', 'Linksaußen', 'Linker Verteidiger', 'Rechtsaußen', 'Linkes Mittelfeld', 'Sturm', 'Rechtes Mittelfeld', 'Mittelfeld', 'Abwehr', 'zuletzt bei Pogon Staszow'])

In [97]:
for player in transfermarkt_data_json:
    if player.position == 'zuletzt bei Pogon Staszow':
        print(player.name)

Hubert Laskowski


In [98]:
for player in transfermarkt_data_json:
    if player.name == 'Hubert Laskowski':
        player.position = 'Rechtsaußen'

In [100]:
for player in transfermarkt_data_json:
    if player.name == 'Hubert Laskowski':
        print(player)

TransfermarktPlayer(player_id=381720, name='Hubert Laskowski', profile_url='https://www.transfermarkt.de/hubert-laskowski/profil/spieler/381720', date_of_birth=None, position='Rechtsaußen', transfers=[Transfer(season='18/19', date='01.01.2019', left='Pogon Staszow', joined='Karriereende', mv='-', fee='-'), Transfer(season='18/19', date='01.07.2018', left='Eintr. Elster', joined='Pogon Staszow', mv='-', fee='ablösefrei'), Transfer(season='17/18', date='01.01.2018', left='Partyzant', joined='Eintr. Elster', mv='-', fee='ablösefrei'), Transfer(season='17/18', date='01.08.2017', left='Korona Kielce', joined='Partyzant', mv='50 Tsd. €', fee='ablösefrei'), Transfer(season='16/17', date='30.06.2017', left='Spartakus Daleszyce', joined='Korona Kielce', mv='50 Tsd. €', fee='Leih-Ende'), Transfer(season='16/17', date='10.02.2017', left='Korona Kielce', joined='Spartakus Daleszyce', mv='50 Tsd. €', fee='Leihe'), Transfer(season='16/17', date='08.12.2016', left='Kotwica', joined='Korona Kielce', m

In [101]:
dm._save_json(transfermarkt_data_json, '../data/transfermarkt_player_data.json')

File saved succesfully. Path: ../data/transfermarkt_player_data.json


#### club, position, price normalization

In [23]:
import pandas as pd

In [53]:
t_to_pq = dm._load_json('../data/transfermarkt_player_data.json')

In [54]:
t_dict =  {'ID 90minut' : [],
           'ID transfermarkt' : [], 
           'Imie i Nazwisko' : [], 
           'Pozycja' : [], 
           'Transfer: sezon' : [], 
           'Transfer: data' : [],
           'Transfer: z klubu' : [], 
           'Transfer: do klubu' : [], 
           'Transfer: wartość rynkowa' : [], 
           'Transfer: cena' : []}

In [55]:
position_dict_mapping = {
'Abwehr' : 'Obrona',
'Defensives Mittelfeld' : 'Defensywny pomocnik',
'Innenverteidiger' : 'Środkowy obrońca',
'Linker Verteidiger' : 'Lewy obrońca',
'Linkes Mittelfeld' : 'Lewy pomocnik',
'Linksaußen' : 'Lewy skrzydłowy',
'Mittelfeld' : 'Pomocnik',
'Mittelstürmer' : 'Środkowy napastnik',
'Offensives Mittelfeld' : 'Ofensywny pomocnik',
'Rechter Verteidiger' : 'Prawy obrońca',
'Rechtes Mittelfeld' : 'Prawy pomocnik',
'Rechtsaußen' : 'Prawy skrzydłowy',
'Sturm' : 'Napastnik',
'Torwart' : 'Bramkarz',
'Zentrales Mittelfeld' : 'Środkowy pomocnik'
}

In [56]:
club_mapping = {
'Arka Gdynia' : 'Arka Gdynia',
'Nieciecza' : 'Bruk-Bet Termalica Nieciecza',
'Cracovia' : 'Cracovia',
'GKS Belchatow' : 'GKS Bełchatów',
'GKS Katowice' : 'GKS Katowice',
'Gornik Leczna' : 'Górnik Łęczna',
'Górnik Zabrze' : 'Górnik Zabrze',
'Jagiellonia' : 'Jagiellonia Białystok',
'Korona Kielce' : 'Korona Kielce',
'Lech Posen' : 'Lech Poznań',
'Lechia Gdansk' : 'Lechia Gdańsk',
'Legia Warschau' : 'Legia Warszawa',
'LKS Lodz' : 'ŁKS Łódź',
'Miedz Legnica' : 'Miedź Legnica',
'Motor Lublin' : 'Motor Lublin',
'Piast Gliwice' : 'Piast Gliwice',
'Podbeskidzie' : 'Podbeskidzie Bielsko-Biała',
'Pogon Stettin' : 'Pogoń Szczecin',
'P. Niepolomice' : 'Puszcza Niepołomice',
'Radomiak' : 'Radomiak Radom',
'Raków' : 'Raków Częstochowa',
'Ruch Chorzow' : 'Ruch Chorzów',
'Sandecja' : 'Sandecja Nowy Sącz',
'Stal Mielec' : 'Stal Mielec',
'Slask Wroclaw' : 'Śląsk Wrocław',
'Nieciecza' : 'Termalica Bruk-Bet Nieciecza',
'Warta Posen' : 'Warta Poznań',
'Widzew Lodz' : 'Widzew Łódź',
'Wisla Krakau' : 'Wisła Kraków',
'Wisla Plock' : 'Wisła Płock',
'Zaglebie Lubin' : 'Zagłębie Lubin',
'Z. Sosnowiec' : 'Zagłębie Sosnowiec',
'Zawisza' : 'Zawisza Bydgoszc'
}

In [57]:
def price_norm(txt):
    txt = txt.replace('</i>', '')
    txt = txt.replace('€', '').strip()
    if 'Leihgebühr:<br /><i class="normaler-text">' in txt:
        txt = 'wypożyczenie'
    if 'Mio.' in txt:
        price, nominator = txt.split()
        price_float = float(price.replace(',', '.'))
        txt = str(int(price_float*1000000))
    if 'Tsd.' in txt:
        price, nominator = txt.split()
        price_float = float(price.replace(',', '.'))
        txt = str(int(price_float*1000))
    if txt == 'ablösefrei':
        txt = 'za darmo'
    if txt == 'Leih-Ende':
        txt = 'koniec wypożyczenia'
    if txt == 'Leihe':
        txt = 'wypożyczenie'
    return txt

In [58]:
def season_parse(txt):
    start, end = txt.split('/')
    new_start = '20'+start
    new_end = '20'+end
    new_txt = new_start+r'/'+new_end
    return new_txt

In [59]:
for player in t_to_pq:
    for transfer in player.transfers:
        t_dict['ID 90minut'].append(player.id_90minut)
        t_dict['ID transfermarkt'].append(player.player_id)
        t_dict['Imie i Nazwisko'].append(player.name)
        t_dict['Pozycja'].append(position_dict_mapping[player.position])
        t_dict['Transfer: sezon'].append(season_parse(transfer.season))
        t_dict['Transfer: data'].append(transfer.date)
        if transfer.left in club_mapping.keys():
            t_dict['Transfer: z klubu'].append(club_mapping[transfer.left])
        else:
            t_dict['Transfer: z klubu'].append(transfer.left)
        if transfer.joined in club_mapping.keys():
            t_dict['Transfer: do klubu'].append(club_mapping[transfer.joined])
        else:
            t_dict['Transfer: do klubu'].append(transfer.joined)
        t_dict['Transfer: wartość rynkowa'].append(price_norm(transfer.mv))
        t_dict['Transfer: cena'].append(price_norm(transfer.fee))

In [60]:
for k,v in t_dict.items():
    print(f'{k}: {len(v)}')

ID 90minut: 5955
ID transfermarkt: 5955
Imie i Nazwisko: 5955
Pozycja: 5955
Transfer: sezon: 5955
Transfer: data: 5955
Transfer: z klubu: 5955
Transfer: do klubu: 5955
Transfer: wartość rynkowa: 5955
Transfer: cena: 5955


In [61]:
t_pd = pd.DataFrame(t_dict)

In [62]:
t_pd.head()

,ID 90minut,ID transfermarkt,Imie i Nazwisko,Pozycja,Transfer: sezon,Transfer: data,Transfer: z klubu,Transfer: do klubu,Transfer: wartość rynkowa,Transfer: cena
0,12027,158798,Piotr Mroziński,Prawy obrońca,2021/2022,15.07.2021,Stalowa Wola,Puszcza Niepołomice,-,za darmo
1,12027,158798,Piotr Mroziński,Prawy obrońca,2018/2019,17.07.2018,Pogon Siedlce,Stalowa Wola,75000,za darmo
2,12027,158798,Piotr Mroziński,Prawy obrońca,2016/2017,04.01.2017,Wisła Płock,Pogon Siedlce,150000,za darmo
3,12027,158798,Piotr Mroziński,Prawy obrońca,2015/2016,06.07.2015,Widzew Łódź,Wisła Płock,200000,za darmo
4,12027,158798,Piotr Mroziński,Prawy obrońca,2012/2013,30.06.2013,Sandecja Nowy Sącz,Widzew Łódź,200000,koniec wypożyczenia


In [63]:
t_pd.to_parquet('../data/transfermarkt_player_data_final_v2.parquet', engine = 'fastparquet')